In [1]:
debug = False
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
import numpy as np
import os
import sys
import time
import datetime
import glob
sys.path.append('../py/')
from s027_kfold_ods import ods_kfold
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import logger_func, get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature, impute_feature
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split


#========================================================================
# Args
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term', 'no_out_flg']
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
#========================================================================

2019-02-18 21:10:53,063 utils 400 [INFO]    [logger_func] start 


In [2]:
df_hist = pd.read_csv('../input/historical_transactions.csv')
df_hist['purchase_amount_new'] = np.round(df_hist['purchase_amount'] / 0.00150265118 + 497.06, 2)

In [5]:
id_list = utils.read_pkl_gzip('../input/0217_merchant_one_id_list.gz')
df_hist.loc[df_hist[key].isin(id_list), :].head(10)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,purchase_amount_new
4163811,Y,C_ID_ba26c0f556,-1,Y,1,B,661,M_ID_fc7d7969c3,-1,-0.717005,2018-01-21 10:50:35,NaN,-1,8,19.9
4163812,Y,C_ID_ba26c0f556,-1,Y,1,B,661,M_ID_fc7d7969c3,-3,-0.717005,2017-11-21 10:49:52,NaN,-1,8,19.9
4163813,Y,C_ID_ba26c0f556,-1,Y,1,B,661,M_ID_fc7d7969c3,0,-0.717005,2018-02-21 08:43:04,NaN,-1,8,19.9
4163814,Y,C_ID_ba26c0f556,-1,Y,1,B,661,M_ID_fc7d7969c3,-2,-0.717005,2017-12-21 08:44:22,NaN,-1,8,19.9
4163815,Y,C_ID_ba26c0f556,-1,Y,1,B,661,M_ID_fc7d7969c3,-5,-0.717005,2017-09-21 11:22:47,NaN,-1,8,19.9
4163816,Y,C_ID_ba26c0f556,-1,Y,1,B,661,M_ID_fc7d7969c3,-7,-0.745405,2017-07-21 21:11:49,NaN,-1,8,1.0
4163817,Y,C_ID_ba26c0f556,-1,Y,1,B,661,M_ID_fc7d7969c3,-6,-0.717005,2017-08-21 08:13:08,NaN,-1,8,19.9
4163818,Y,C_ID_ba26c0f556,-1,Y,1,B,661,M_ID_fc7d7969c3,-4,-0.717005,2017-10-21 09:38:01,NaN,-1,8,19.9
4233736,Y,C_ID_7f628a65cd,-1,Y,1,B,661,M_ID_fc7d7969c3,-8,-0.745405,2017-06-10 20:58:16,NaN,-1,8,1.0
4233737,Y,C_ID_7f628a65cd,-1,Y,1,B,661,M_ID_fc7d7969c3,-1,-0.745405,2018-01-14 16:59:52,NaN,-1,8,1.0


In [7]:
use_cols = [key, 'month_lag', 'purchase_date', 'authorized_flag', 'installments', 'merchant_id', 'purchase_amount_new', 'subsector_id', 'merchant_category_id']
df_hist = df_hist[use_cols]

In [23]:
#========================================================================
# 最初の購買と次の購買の比
df_hist.set_index([key, 'merchant_id'], inplace=True)
# df_hist.sort_values(by=[key, 'purchase_date', 'merchant_id'], inplace=True)
# df_hist = utils.row_number(df=df_hist, level=[key, 'merchant_id'])
# df_amt = df_hist[[key, 'merchant_id', 'purchase_date', 'purchase_amount_new']]
#========================================================================

In [40]:
# df_hist.sort_values(by=[key, 'purchase_date', 'merchant_id'], inplace=True)
# df_hist = utils.row_number(df=df_hist, level=[key, 'merchant_id'])
# tmp = df_hist[df_hist[key]=='C_ID_00007093c1']
df_hist = utils.row_number(df=df_hist, level=[key, 'merchant_id'])
df_hist.sort_values(by=[key, 'merchant_id', 'purchase_date'], inplace=True)
df_hist

,card_id,merchant_id,month_lag,purchase_date,authorized_flag,installments,purchase_amount_new,subsector_id,merchant_category_id,row_no
102,C_ID_00007093c1,M_ID_0379adb435,-8,2017-06-07 10:28:34,Y,1,56.00,18,45,1
103,C_ID_00007093c1,M_ID_0379adb435,-7,2017-07-06 12:41:24,Y,2,151.00,18,45,2
104,C_ID_00007093c1,M_ID_0379adb435,-6,2017-08-08 15:26:32,Y,2,65.00,18,45,3
105,C_ID_00007093c1,M_ID_0379adb435,-5,2017-09-25 12:14:35,Y,3,220.00,18,45,4
148,C_ID_00007093c1,M_ID_06a8d84366,0,2018-02-07 11:11:27,Y,1,207.44,15,437,1
106,C_ID_00007093c1,M_ID_0dabc5a70c,-8,2017-06-07 10:47:20,Y,2,98.00,10,298,1
107,C_ID_00007093c1,M_ID_0dabc5a70c,-5,2017-09-28 14:39:39,Y,3,149.50,10,298,2
95,C_ID_00007093c1,M_ID_109e9d2b1d,-9,2017-05-16 17:16:11,Y,1,106.00,16,367,1
96,C_ID_00007093c1,M_ID_109e9d2b1d,-7,2017-07-05 17:27:26,Y,1,39.00,16,367,2
97,C_ID_00007093c1,M_ID_15446e939c,-9,2017-05-18 16:52:22,N,1,702.00,34,108,1


In [11]:
df_rank2 = df_amt[df_amt['row_no']<=2]
df_rank2

df_rank2.set_index([key, 'merchant_id', 'row_no']).unstack()

,card_id,merchant_id,purchase_date,purchase_amount_new,row_no
0,C_ID_00007093c1,M_ID_0379adb435,2017-06-07 10:28:34,56.00,1
1,C_ID_00007093c1,M_ID_0379adb435,2017-07-06 12:41:24,151.00,1
2,C_ID_00007093c1,M_ID_0379adb435,2017-08-08 15:26:32,65.00,1
3,C_ID_00007093c1,M_ID_0379adb435,2017-09-25 12:14:35,220.00,1
4,C_ID_00007093c1,M_ID_06a8d84366,2018-02-07 11:11:27,207.44,1
5,C_ID_00007093c1,M_ID_0dabc5a70c,2017-06-07 10:47:20,98.00,1
6,C_ID_00007093c1,M_ID_0dabc5a70c,2017-09-28 14:39:39,149.50,1
7,C_ID_00007093c1,M_ID_109e9d2b1d,2017-05-16 17:16:11,106.00,1
8,C_ID_00007093c1,M_ID_109e9d2b1d,2017-07-05 17:27:26,39.00,1
9,C_ID_00007093c1,M_ID_15446e939c,2017-05-18 16:52:22,702.00,1
